## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-14-16-50-39 +0000,startribune,Brooks: Movie crew at the State Fair is making...,https://www.startribune.com/the-minnesota-stat...
1,2025-08-14-16-49-13 +0000,cbc,Poilievre says pushing Liberals to repeal EV m...,https://www.cbc.ca/news/politics/poilievre-fal...
2,2025-08-14-16-49-00 +0000,nypost,"2,000-year-old coin produced by Jewish rebel u...",https://nypost.com/2025/08/14/lifestyle/2000-y...
3,2025-08-14-16-48-01 +0000,nypost,Obama’s newly revealed call to Zohran Mamdani ...,https://nypost.com/2025/08/14/us-news/obamas-n...
4,2025-08-14-16-47-29 +0000,nyt,Pilgrimage Route Hit as Flash Flood Kills Doze...,https://www.nytimes.com/2025/08/14/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
75,trump,49
228,putin,20
336,new,16
425,court,11
415,russia,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
104,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...,119
38,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...,109
301,2025-08-13-17-57-20 +0000,nypost,Trump says he doesn’t believe he can convince ...,https://nypost.com/2025/08/13/us-news/trump-sa...,106
55,2025-08-14-15-07-22 +0000,nyt,Russia Seeks to Add Trade and Arms Control Int...,https://www.nytimes.com/2025/08/14/world/europ...,98
257,2025-08-13-20-57-00 +0000,wsj,Trump Agrees on Ukraine Red Lines With Europe ...,https://www.wsj.com/world/trump-talks-ukraine-...,97


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
104,119,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...
301,37,2025-08-13-17-57-20 +0000,nypost,Trump says he doesn’t believe he can convince ...,https://nypost.com/2025/08/13/us-news/trump-sa...
191,37,2025-08-14-03-00-00 +0000,wsj,As other Asian countries ramp up manufacturing...,https://www.wsj.com/economy/trade/trump-tariff...
14,36,2025-08-14-16-29-11 +0000,nypost,DC cops allowed to alert ICE about illegal imm...,https://nypost.com/2025/08/14/us-news/dc-cops-...
109,34,2025-08-14-11-39-23 +0000,wapo,"Top South Korean court listens to ‘Baby Shark,...",https://www.washingtonpost.com/style/2025/08/1...
111,34,2025-08-14-11-30-00 +0000,nypost,Eric Adams advances fight against NYC public d...,https://nypost.com/2025/08/14/us-news/eric-ada...
230,33,2025-08-13-22-21-27 +0000,nypost,Homeless lunatic accused of killing pregnant w...,https://nypost.com/2025/08/13/world-news/homel...
22,31,2025-08-14-16-19-00 +0000,wsj,Mortgage rates edged down to their lowest leve...,https://www.wsj.com/economy/housing/mortgage-r...
263,30,2025-08-13-20-48-49 +0000,latimes,"They led police on a wild L.A. car chase, then...",https://www.latimes.com/california/story/2025-...
20,29,2025-08-14-16-21-29 +0000,nypost,Navy SEAL almost killed in harrowing firefight...,https://nypost.com/2025/08/14/lifestyle/war-he...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
